In [1]:
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from dataWrangler import *
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
import pandas as pd
from scrape_craigslist import Scrape
from predictions import *
import urllib.request
import warnings
warnings.filterwarnings('ignore')
import requests
import datetime
import math
import glob   
from xgboost import XGBRegressor
import os
import io
import re
import numpy as np

In [2]:
li=[]
for filename in glob.glob(r"..\UsedCar-PriceEstimator\data\*.csv"):
    collection = pd.read_csv(filename, index_col=None, header=0)
    li.append(collection)
frame = pd.concat(li, axis=0, ignore_index=True)


In [3]:
frame = pd.concat(li, axis=0, ignore_index=True)

In [81]:
Link='https://sfbay.craigslist.org/eby/cto/d/discovery-bay-1969-camaro-ss/7339283378.html'
lis=[]

link=[Link]
raw=Scrape(link)
df1=Spell(raw)
df2=Replace(df1)
df3=Process(df2)
inputcar=df3.iloc[0,0]
df_spelled=Spell(frame)
df_replaced=Replace(df_spelled)
proccessed=Process(df_replaced,inputcar)
df=Outliers_light(proccessed)
df=delete_outliers(df)

In [82]:
first_len=len(df.columns)

In [83]:
#get_dummies
df['score:'] = df['score:'].astype(int)
df['score:'] = df['score:'].astype(str)
df=pd.get_dummies(df)

In [84]:
second_len=len(df.columns)

In [85]:
first_len,second_len

(5, 8)

In [86]:
X=df.drop(columns='price:')
y=df['price:']
train_features, test_features, train_labels, test_labels = \
train_test_split(X, y, test_size = 0.05, random_state = 3)
#n_estimators down below can be specified
#max_depth

scaler = StandardScaler()
scaler.fit(train_features)
scaler.transform(train_features)

rf =  XGBRegressor(learning_rate=0.31,gamma=1,max_depth=12,
                  eval_metric='mlogloss',n_estimators=100, eta=0.6, 
                   subsample=0.2, colsample_bytree=0.8)

scaler.transform(test_features)
rf.fit(train_features, train_labels);
# Calculate mean absolute percentage error (MAPE)
y_pred=rf.predict(test_features)
errors=np.abs((test_labels - y_pred))
mape = 100 * (errors / test_labels)
accuracy = 100 - np.mean(mape)
print(f'{inputcar} Accuracy:', round(accuracy, 2), '%.')
        
        
if(len(df.columns)>6):
    x_feat=df3.drop(['Make&Model','price:'],axis=1)
if(len(df.columns)<=6):
    x_feat=df3.drop(['Make&Model','cylinders:','price:'],axis=1)
    
x_feat=sample_dummies(x_feat,X)

scaler.transform(x_feat)
y_actual=df3[['price:']]
result="predicted:",int(2**rf.predict(x_feat)[0]),"actual:",y_actual.values[0][0]
result


chevrolet camaro Accuracy: 96.94 %.


('predicted:', 23353, 'actual:', 69500)

In [66]:
def sample_dummies(x_feat,X):
    x_feat['score:_0']=0
    x_feat['score:_1']=0
    x_feat['score:_2']=0
    x_feat['score:_3']=0
    
    for x in x_feat.values:
        if(x[1]==0):
            x_feat['score:_0']=0
        if(x[1]==1):
            x_feat['score:_1']=1
        if(x[1]==2):
            x_feat['score:_2']=1
        if(x[1]==3):
            x_feat['score:_3']=1
    
    extra_columns=list((set(x_feat.columns)-set(X.columns)))
    x_feat.drop(columns=extra_columns,inplace=True)
    x_feat=x_feat.reset_index(drop=True)
    return x_feat

In [ ]:
x_feat

In [ ]:
X.head(1)

In [140]:
rf??